In [1]:
%matplotlib inline

In [2]:
import supervisely_lib as sly
from tqdm import tqdm
import random
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
team_name = 'max'
workspace_name = 'test_dtl_segmentation'

project_name = 'lemons_annotated_detection_005'

validation_portion = 0.1

tag_meta_train = sly.TagMeta('train', sly.TagValueType.NONE)
tag_meta_val = sly.TagMeta('val', sly.TagValueType.NONE)

In [4]:
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [5]:
print("Server address: ", address)
print("Your API token: ", token)

Server address:  http://192.168.1.69:5555
Your API token:  OfaV5z24gEQ7ikv2DiVdYu1CXZhMavU7POtJw2iDtQtvGUux31DUyWTXW6mZ0wd3IRuXTNtMFS9pCggewQWRcqSTUi4EJXzly8kH7MJL1hm3uZeM2MCn5HaoEYwXejKT


In [6]:
api = sly.Api(address, token)

In [7]:
team = api.team.get_info_by_name(team_name)
workspace = api.workspace.get_info_by_name(team.id, workspace_name)
print("Current context: Team {!r}, Workspace {!r}".format(team.name, workspace.name))

Current context: Team 'max', Workspace 'test_dtl_segmentation'


In [12]:
project = api.project.get_info_by_name(workspace.id, project_name)
meta_json = api.project.get_meta(project.id)
meta = sly.ProjectMeta.from_json(meta_json)

In [13]:
def process_meta(input_meta):
    output_meta = input_meta.clone()
    output_meta = output_meta.add_img_tag_meta(tag_meta_train)
    output_meta = output_meta.add_img_tag_meta(tag_meta_val)
    return output_meta

In [15]:
dst_meta = process_meta(meta)
print(dst_meta)
api.project.update_meta(project.id, dst_meta.to_json())

ProjectMeta:

Object Classes:
Name:  lemon_bboxShape: Rectangle    Color: [81, 198, 170]
Name:  kiwi_bbox Shape: Rectangle    Color: [255, 0, 0]
Name:  lemon     Shape: Bitmap       Color: [81, 198, 170]
Name:  kiwi      Shape: Bitmap       Color: [255, 0, 0]

Image Tags
Tag Metas:
Name:  train                    Value type:none          Possible values:None      
Name:  val                      Value type:none          Possible values:None      

Object Tags
Tag Metas:
Empty




In [17]:
print("Project {!r}: train/val tagging (inplace)".format(project.name))
for dataset in api.dataset.get_list(project.id):
    print('Dataset: {}'.format(dataset.name))
    for image in tqdm(api.image.get_list(dataset.id)):
        tag = sly.Tag(tag_meta_val) if random.random() <= validation_portion else sly.Tag(tag_meta_train)
        
        ann_json = api.annotation.download(image.id).annotation
        ann = sly.Annotation.from_json(ann_json, meta)
        ann = ann.add_tag(tag)

        api.annotation.upload(image.id, ann.to_json())

  3%|▎         | 2/72 [00:00<00:04, 16.94it/s]

Project 'lemons_annotated_detection_005': train/val tagging (inplace)
Dataset: ds1


100%|██████████| 72/72 [00:03<00:00, 19.22it/s]
